<a href="https://github.com/PhinanceScientist"><img src = "https://i.ibb.co/NLfc0SV/Deveaner.png" width = 100> </a>
<h1 align=center><font size = 5>Merida Neighbourhoods Clustered by Economic Vulnerability due to the COVID-19 Outbreak</font></h1>

## Introduction

For this project I will be using some prepared data from a postal public web page due to the lack of postal and geodata from Mérida, Yucatán in México. The goal is to obtain some relevant information from the economic vulnerability of neighborhoods from Merida based on the information retrieved by the Foursquare API. k-means will be used to group the neighbourhoods and finally I will use the Folium library to visualize the results.
This approach is an attempt for visualizing the main neighborhoods inside Merida in order to cluster the most economic vulnerable places as the COVID-19 expands.

Please do notice that if you want to render this Jupyter notebook (show the folium maps) you can use this link https://nbviewer.jupyter.org/

## Data

The data needed for this project  can be found on this local postal services web page called <a href="https://www.heraldo.com.mx/"> Heraldo.com.mx </a> where we can find several postal codes from México. In this case we will be focused on <a href="https://www.heraldo.com.mx/yucatan/merida/merida/">Mérida's postal codes</a>.<br>
As for the CSV file used it is based on the first 100 postal codes from Mérida (ascending order starting from the downtown area as common knowledge) and then linked to its own Latitude and Longitude as a result of a Google Maps Search for each one.<br>
The Foursquare's API will be used to retrieve information of the venue on each neighborhood, type of each venue will be our goal to determine how crowded they are and therefore the whole vulnerability of the surrounding area.  

***

## Methodology


## Importing Libraries

In [1]:
#Import requests for web scraping
import pandas as pd
import requests as rq
import numpy as np
import io

import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

print('Libraries installed')

Libraries installed


## First we need to retrieve our data, in this case we will use the file created and hosted on my GitHub repository. <br>
CSV URL File: https://raw.githubusercontent.com/PhinanceScientist/Coursera_Capstone/master/merida_cp_1_prepared.csv

In [2]:
urlCSV = 'https://raw.githubusercontent.com/PhinanceScientist/Coursera_Capstone/master/merida_cp_1_prepared.csv' #Retreive the data
geoSpatial = pd.read_csv(urlCSV) #Turned to dataFrame
newdf = geoSpatial.rename(columns ={"cp":"Postcode","colonia":"Neighbourhood"}) #Rename our column in order to have the same Column title as our previous DataFrame
newdf


Postcode                  Neighbourhood        lat        lon
0       97000      jardines de san sebastian  20.989200 -89.756400
1       97000            privada del maestro  20.982308 -89.626156
2       97000                  merida centro  20.968927 -89.645942
3       97000                      los cocos  20.948595 -89.630134
4       97000  privada garcia gineres c - 29  20.989226 -89.638116
5       97003                      los reyes  20.978651 -89.575990
6       97050                        yucatan  20.994835 -89.628827
7       97050                   alcal martin  20.991941 -89.622019
8       97059                       seoorial  20.990414 -89.621707
9       97060                carrillo ancona  20.985286 -89.643317
10      97068                         itzaes  20.978626 -89.643890
11      97069                     inalmbrica  20.982404 -89.647450
12      97070                 dolores patron  20.989592 -89.626070
13      97070                    el pedregal  20.995481 -89.634048
14      97070                 garcia gineres  20.989879 -89.635259
15      97080                      la huerta  20.983163 -89.615553
16      97088                  santa cecilia  20.981542 -89.615670
17      97089                        cupules  20.996456 -89.632023
18      97098                        lourdes  20.960203 -89.610744
19      97099                          waspa  20.969501 -89.608684
20      97100                      itzimna 2  20.992752 -89.614184
21      97100                        itzimna  20.992752 -89.614184
22      97100              rinconada itzmina  20.992707 -89.606591
23      97108                  las arboledas  20.981326 -89.608092
24      97109                 jes䚂s carranza  20.986134 -89.612606
25      97109                ferrocarrileros  20.981507 -89.610899
26      97113                      xaman-tan  21.033779 -89.604826
27      97113                    san antonio  20.988620 -89.658706
28      97113                     montebello  21.030032 -89.599605
29      97113           gran royal altabrisa  21.023328 -89.589453
..        ...                            ...        ...        ...
636     97314                  gran santa fe  21.026500 -89.676500
637     97314                  ciudad caucel  21.026500 -89.676500
638     97314                    balcones ii  21.026500 -89.676500
639     97314             jardines de caucel  21.026500 -89.676500
640     97314                 hogares caucel  21.026500 -89.676500
641     97314                       arboleda  21.026500 -89.676500
642     97314         jardines del caucel ii  21.026500 -89.676500
643     97314           jardines de poniente  21.026500 -89.676500
644     97314                   las torres i  21.026500 -89.676500
645     97314                  las torres ii  21.026500 -89.676500
646     97314                   villa jardin  21.026500 -89.676500
647     97314               villas de caucel  21.026500 -89.676500
648     97314                    viva caucel  21.026500 -89.676500
649     97314                         sasula  21.026500 -89.676500
650     97315               san pedro chimay  20.864600 -89.579600
651     97315                          molas  20.816700 -89.630000
652     97315             san antonio tzacal  20.751700 -89.654500
653     97315                     texn cmara  20.789800 -89.666300
654     97315                       dzununcn  20.864700 -89.653800
655     97315                          petac  20.769600 -89.658600
656     97315                       xmatkuil  20.860300 -89.625500
657     97315                      dzununcan  20.864700 -89.653800
658     97315                     hunxectamn  20.883500 -89.558100
659     97315                   tahdzibichen  20.885900 -89.597700
660     97315           santa cruz palomeque  20.880700 -89.653300
661     97315       nuevo san jose tecoh iii  20.838000 -89.632600
662     97315           nuevo san jose tecoh  20.838000 -89.632600
663     97316                      dzoyaxche  20.787

In [43]:
newdf.drop_duplicates(subset='Postcode', keep="first")#Dropping duplicate postcodes and keeping only the first value

newdf.head()

Postcode                  Neighbourhood        lat        lon
0     97000      jardines de san sebastian  20.989200 -89.756400
1     97000            privada del maestro  20.982308 -89.626156
2     97000                  merida centro  20.968927 -89.645942
3     97000                      los cocos  20.948595 -89.630134
4     97000  privada garcia gineres c - 29  20.989226 -89.638116

In [42]:

newdf = newdf[:100]

***

***

## First we need to retrieve our data, in this case we will use the file given by the instructions <br>
CSV URL File: https://raw.githubusercontent.com/PhinanceScientist/Coursera_Capstone/master/merida_cp_prepared.csv

In [5]:
urlCSV = 'https://raw.githubusercontent.com/PhinanceScientist/Coursera_Capstone/master/merida_cp_1_prepared.csv' #Retreive the data
geoSpatial = pd.read_csv(urlCSV) #Turned to dataFrame
newdf = geoSpatial.rename(columns ={"cp":"Postcode","colonia":"Neighbourhood"}) #Rename our column in order to have the same Column title as our previous DataFrame
newdf


Postcode                  Neighbourhood        lat        lon
0       97000      jardines de san sebastian  20.989200 -89.756400
1       97000            privada del maestro  20.982308 -89.626156
2       97000                  merida centro  20.968927 -89.645942
3       97000                      los cocos  20.948595 -89.630134
4       97000  privada garcia gineres c - 29  20.989226 -89.638116
5       97003                      los reyes  20.978651 -89.575990
6       97050                        yucatan  20.994835 -89.628827
7       97050                   alcal martin  20.991941 -89.622019
8       97059                       seoorial  20.990414 -89.621707
9       97060                carrillo ancona  20.985286 -89.643317
10      97068                         itzaes  20.978626 -89.643890
11      97069                     inalmbrica  20.982404 -89.647450
12      97070                 dolores patron  20.989592 -89.626070
13      97070                    el pedregal  20.995481 -89.634048
14      97070                 garcia gineres  20.989879 -89.635259
15      97080                      la huerta  20.983163 -89.615553
16      97088                  santa cecilia  20.981542 -89.615670
17      97089                        cupules  20.996456 -89.632023
18      97098                        lourdes  20.960203 -89.610744
19      97099                          waspa  20.969501 -89.608684
20      97100                      itzimna 2  20.992752 -89.614184
21      97100                        itzimna  20.992752 -89.614184
22      97100              rinconada itzmina  20.992707 -89.606591
23      97108                  las arboledas  20.981326 -89.608092
24      97109                 jes䚂s carranza  20.986134 -89.612606
25      97109                ferrocarrileros  20.981507 -89.610899
26      97113                      xaman-tan  21.033779 -89.604826
27      97113                    san antonio  20.988620 -89.658706
28      97113                     montebello  21.030032 -89.599605
29      97113           gran royal altabrisa  21.023328 -89.589453
..        ...                            ...        ...        ...
636     97314                  gran santa fe  21.026500 -89.676500
637     97314                  ciudad caucel  21.026500 -89.676500
638     97314                    balcones ii  21.026500 -89.676500
639     97314             jardines de caucel  21.026500 -89.676500
640     97314                 hogares caucel  21.026500 -89.676500
641     97314                       arboleda  21.026500 -89.676500
642     97314         jardines del caucel ii  21.026500 -89.676500
643     97314           jardines de poniente  21.026500 -89.676500
644     97314                   las torres i  21.026500 -89.676500
645     97314                  las torres ii  21.026500 -89.676500
646     97314                   villa jardin  21.026500 -89.676500
647     97314               villas de caucel  21.026500 -89.676500
648     97314                    viva caucel  21.026500 -89.676500
649     97314                         sasula  21.026500 -89.676500
650     97315               san pedro chimay  20.864600 -89.579600
651     97315                          molas  20.816700 -89.630000
652     97315             san antonio tzacal  20.751700 -89.654500
653     97315                     texn cmara  20.789800 -89.666300
654     97315                       dzununcn  20.864700 -89.653800
655     97315                          petac  20.769600 -89.658600
656     97315                       xmatkuil  20.860300 -89.625500
657     97315                      dzununcan  20.864700 -89.653800
658     97315                     hunxectamn  20.883500 -89.558100
659     97315                   tahdzibichen  20.885900 -89.597700
660     97315           santa cruz palomeque  20.880700 -89.653300
661     97315       nuevo san jose tecoh iii  20.838000 -89.632600
662     97315           nuevo san jose tecoh  20.838000 -89.632600
663     97316                      dzoyaxche  20.787

In [6]:
newdf.drop_duplicates(subset='Postcode', keep="first")#Dropping duplicate postcodes and keeping only the first value
newdf.head()

Postcode                  Neighbourhood        lat        lon
0     97000      jardines de san sebastian  20.989200 -89.756400
1     97000            privada del maestro  20.982308 -89.626156
2     97000                  merida centro  20.968927 -89.645942
3     97000                      los cocos  20.948595 -89.630134
4     97000  privada garcia gineres c - 29  20.989226 -89.638116

In [44]:

newdf = newdf[:100]

### We need to import our libraries for visualization

In [8]:
!pip -q install folium
import folium
print('Folium imported')

Folium imported


## 1. Exploring the dataset


In [9]:
map_merida = folium.Map(location=[20.97537, -89.61696], zoom_start=11) # Create Map

# add markers to map
for lat, lng, borough, neighborhood in zip(newdf['lat'], newdf['lon'], newdf['Postcode'], newdf['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_merida) 
    
map_merida


### Adding the Foursquare credentials

In [10]:
#@hidden_cell
CLIENT_ID = 'DGBSOBI1JYHOTEEC5WQBC41VJNTTUGDB0IJH4U4GI5HITY4D' # your Foursquare ID
CLIENT_SECRET = 'NDTXJZISJVIJX0J5V5RSDHXJULPWBBI2ND2EN3JH11ULSJQO' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: DGBSOBI1JYHOTEEC5WQBC41VJNTTUGDB0IJH4U4GI5HITY4D
CLIENT_SECRET:NDTXJZISJVIJX0J5V5RSDHXJULPWBBI2ND2EN3JH11ULSJQO


### For this excercise we will use only the first 100 Neighbourhoods as they are the most commercial like

In [11]:

newdf.head()

Postcode                  Neighbourhood        lat        lon
0     97000      jardines de san sebastian  20.989200 -89.756400
1     97000            privada del maestro  20.982308 -89.626156
2     97000                  merida centro  20.968927 -89.645942
3     97000                      los cocos  20.948595 -89.630134
4     97000  privada garcia gineres c - 29  20.989226 -89.638116

In [12]:
neighborhood_latitude = newdf.loc[1, 'lat'] # neighborhood latitude value
neighborhood_longitude = newdf.loc[1, 'lon'] # neighborhood longitude value

neighborhood_name = newdf.loc[1, 'Neighbourhood'] # neighbourhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of privada del maestro are 20.9823085, -89.62615579999999.


### Let's create the GET request URL. 

In [13]:


LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=DGBSOBI1JYHOTEEC5WQBC41VJNTTUGDB0IJH4U4GI5HITY4D&client_secret=NDTXJZISJVIJX0J5V5RSDHXJULPWBBI2ND2EN3JH11ULSJQO&v=20180605&ll=20.9823085,-89.62615579999999&radius=500&limit=100'

### Send the GET request and examine the resutls

In [14]:
results = rq.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e9258359388d70028088d9a'},
 'response': {'headerLocation': 'Mérida',
  'headerFullLocation': 'Mérida',
  'headerLocationGranularity': 'city',
  'totalResults': 23,
  'suggestedBounds': {'ne': {'lat': 20.986808504500004,
    'lng': -89.62134521245169},
   'sw': {'lat': 20.977808495499993, 'lng': -89.6309663875483}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5149e0bde4b008ba38e2912d',
       'name': 'Bistro Cultural',
       'location': {'address': 'C. 66 Centro',
        'crossStreet': '43',
        'lat': 20.978596857641357,
        'lng': -89.62548953628293,
        'labeledLatLngs': [{'label': 'display',
          'lat': 20.978596857641357,
          'lng': -89.62548953628293}],
        'distance': 418,
        'postalCode': 

### Function that extracts the category of the venue

In [15]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

### Now we clean the json and structure it into a pandas dataframe.

## 2. Exploring Neighbourhoods in Merida 

### Function to repeat the same process to all the neighbourhoods in Merida

In [16]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = rq.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### The code to run the above function on each neighborhood and create a new dataframe called *dt_Merida_venues*.

In [17]:

dt_Merida_venues = getNearbyVenues(names=newdf['Neighbourhood'],
                                   latitudes=newdf['lat'],
                                   longitudes=newdf['lon']
                                  )

jardines de san sebastian
privada del maestro
merida centro
los cocos
privada garcia gineres c - 29
los reyes
yucatan
alcal martin
seoorial
carrillo ancona
itzaes
inalmbrica
dolores patron
el pedregal
garcia gineres
la huerta
santa cecilia
cupules
lourdes
waspa
itzimna 2
itzimna
rinconada itzmina
las arboledas
jes䚂s carranza
ferrocarrileros
xaman-tan
san antonio
montebello
gran royal altabrisa
sol campestre
monte alban
privada monterreal plus
hacienda dzodzil
cordemex
gonzalo guerrero
residencial san angelo
montes de ame
san antonio cucul
san ramon norte
plan de ayala
villas del sol
villas la hacienda
benito jurez nte
gonzalo guerrero
tecnolugico
campestre
del norte
centro sct yucatn
privada nuevo mexico
montejo
buenavista
privada mediterrneo
residencial colonia mexico
privada real mexico
mexico norte
emiliano zapata nte
vista alegre
san remo
san carlos
privada vista alegre
montecarlo norte
diaz ordaz
montecarlo
privada maya
altabrisa
cumbres de altabrisa
missan ii
residencial palmeral

### Check the size of the new dataFrame


In [18]:
print(dt_Merida_venues.shape)
dt_Merida_venues.head()

(2257, 7)


Neighbourhood  Neighbourhood Latitude  Neighbourhood Longitude  \
0  privada del maestro               20.982308               -89.626156   
1  privada del maestro               20.982308               -89.626156   
2  privada del maestro               20.982308               -89.626156   
3  privada del maestro               20.982308               -89.626156   
4  privada del maestro               20.982308               -89.626156   

                                       Venue  Venue Latitude  Venue Longitude  \
0                            Bistro Cultural       20.978597       -89.625490   
1                    Centro Cultural Ibérica       20.982269       -89.628082   
2  Restaurant Reforma "El popular Soberanis"       20.985473       -89.624678   
3                           Hotel Casa Nobel       20.980803       -89.626846   
4                             El Tío Ricardo       20.986303       -89.627185   

       Venue Category  
0                Café  
1        Concert Hall  
2          Restaurant  
3     Bed & Breakfast  
4  Mexican Restaurant

### Let's group our dataframe by Neighbourhood and count how many venues they have

In [19]:
dt_Merida_venues.groupby('Neighbourhood').count()

Neighbourhood Latitude  \
Neighbourhood                                                  
alcal martin                                               8   
altabrisa                                                 64   
benito jurez nte                                          26   
buenavista                                                15   
camara de comercio norte                                  19   
campestre                                                 23   
carrillo ancona                                           26   
centro sct yucatn                                         17   
cordemex                                                  38   
cumbres de altabrisa                                       6   
cupules                                                   13   
del norte                                                 18   
diaz ordaz                                                11   
dolores patron                                            29   
el arco                                                   32   
el pedregal                                               20   
emiliano zapata nte                                       27   
felipe carrillo puerto nte                                18   
ferrocarrileros                                            6   
garcia gineres                                            21   
gonzalo guerrero                                         132   
gran royal altabrisa                                       5   
hacienda dzodzil                                          11   
inalmbrica                                                11   
itzaes                                                    15   
itzimna                                                   39   
itzimna 2                                                 39   
jardines de merida                                        31   
jardines de vista alegre                                  43   
jardines de vista alegre ii                               32   
...                                                      ...   
privada villa cholul                                       5   
privada vista alegre                                      16   
residencial camara de comercio norte                      19   
residencial colonia mexico                                45   
residencial del arco                                      38   
residencial palmerales de altabrisa                        1   
residencial san angelo                                     6   
rinconada itzmina                                         29   
san antonio                                               16   
san antonio cinta                                          4   
san antonio cucul                                         62   
san carlos                                                37   
san pedro cholul                                          11   
san ramon norte                                           20   
san remo                                                  11   
santa cecilia                                             42   
santa maria                                               26   
santa rita cholul                                          6   
seoorial                                                  16   
sol campestre                                             32   
tecnolugico                                               15   
torremolinos                                               7   
villas del sol                                            10   
villas la hacienda                                        46   
vista alegre                                              27   
vista alegre lotificacion                                  4   
vista alegre norte                                        28   
waspa                                                      8   
xaman-tan                                                 53   
yucatan                                                   18   

     

In [20]:
# Unique venues categories
print('There are {} uniques categories.'.format(len(dt_Merida_venues['Venue Category'].unique())))

There are 215 uniques categories.


## 3. Analyze Each Neighbourhood

In [21]:
# one hot encoding
dt_Merida_onehot = pd.get_dummies(dt_Merida_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
dt_Merida_onehot['Neighbourhood'] = dt_Merida_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [dt_Merida_onehot.columns[-1]] + list(dt_Merida_onehot.columns[:-1])
dt_Merida_onehot = dt_Merida_onehot[fixed_columns]

dt_Merida_onehot.head()

Neighbourhood  ATM  Accessories Store  American Restaurant  \
0  privada del maestro    0                  0                    0   
1  privada del maestro    0                  0                    0   
2  privada del maestro    0                  0                    0   
3  privada del maestro    0                  0                    0   
4  privada del maestro    0                  0                    0   

   Argentinian Restaurant  Art Gallery  Art Museum  Arts & Crafts Store  \
0                       0            0           0                    0   
1                       0            0           0                    0   
2                       0            0           0                    0   
3                       0            0           0                    0   
4                       0            0           0                    0   

   Arts & Entertainment  Asian Restaurant  ...  Vegetarian / Vegan Restaurant  \
0                     0                 0  ...                              0   
1                     0                 0  ...                              0   
2                     0                 0  ...                              0   
3                     0                 0  ...                              0   
4                     0                 0  ...                              0   

   Video Game Store  Video Store  Warehouse Store  Whisky Bar  Wine Bar  \
0                 0            0                0           0         0   
1                 0            0                0           0         0   
2                 0            0                0           0         0   
3                 0            0                0           0         0   
4                 0            0                0           0         0   

   Wings Joint  Women's Store  Yoga Studio  Yucatecan Restaurant  
0            0              0            0                     0  
1            0              0            0                     0  
2            0              0            0                     0  
3            0              0            0                     0  
4            0              0            0                     0  

[5 rows x 216 columns]

In [22]:
dt_Merida_onehot.shape

(2257, 216)

### Next, let's group rows by neighbourhood and by taking the mean of the frequency of occurrence of each category

In [23]:
dt_Merida_grouped = dt_Merida_onehot.groupby('Neighbourhood').mean().reset_index()
dt_Merida_grouped

Neighbourhood       ATM  Accessories Store  \
0                           alcal martin  0.000000           0.000000   
1                              altabrisa  0.000000           0.000000   
2                       benito jurez nte  0.000000           0.000000   
3                             buenavista  0.000000           0.000000   
4               camara de comercio norte  0.000000           0.000000   
5                              campestre  0.000000           0.000000   
6                        carrillo ancona  0.000000           0.000000   
7                      centro sct yucatn  0.000000           0.000000   
8                               cordemex  0.000000           0.000000   
9                   cumbres de altabrisa  0.000000           0.000000   
10                               cupules  0.000000           0.000000   
11                             del norte  0.000000           0.000000   
12                            diaz ordaz  0.000000           0.000000   
13                        dolores patron  0.000000           0.000000   
14                               el arco  0.000000           0.000000   
15                           el pedregal  0.000000           0.000000   
16                   emiliano zapata nte  0.000000           0.000000   
17            felipe carrillo puerto nte  0.000000           0.000000   
18                       ferrocarrileros  0.000000           0.000000   
19                        garcia gineres  0.000000           0.000000   
20                      gonzalo guerrero  0.000000           0.000000   
21                  gran royal altabrisa  0.000000           0.000000   
22                      hacienda dzodzil  0.000000           0.000000   
23                            inalmbrica  0.000000           0.000000   
24                                itzaes  0.000000           0.000000   
25                               itzimna  0.000000           0.000000   
26                             itzimna 2  0.000000           0.000000   
27                    jardines de merida  0.000000           0.000000   
28              jardines de vista alegre  0.000000           0.000000   
29           jardines de vista alegre ii  0.000000           0.000000   
..                                   ...       ...                ...   
61                  privada villa cholul  0.000000           0.000000   
62                  privada vista alegre  0.000000           0.000000   
63  residencial camara de comercio norte  0.000000           0.000000   
64            residencial colonia mexico  0.000000           0.000000   
65                  residencial del arco  0.000000           0.000000   
66   residencial palmerales de altabrisa  0.000000           0.000000   
67                residencial san angelo  0.000000           0.000000   
68                     rinconada itzmina  0.000000           0.000000   
69                           san antonio  0.000000           0.000000   
70                     san antonio cinta  0.000000           0.000000   
71                     san antonio cucul  0.000000           0.000000   
72                            san carlos  0.000000           0.000000   
73                      san pedro cholul  0.000000           0.000000   
74                       san ramon norte  0.000000           0.000000   
75                              san remo  0.000000           0.000000   
76                         santa cecilia  0.000000           0.000000   
77                           santa maria  0.000000           0.000000   
78                     santa rita cholul  0.166667           0.000000   
79                              seoorial  0.000000           0.000000   
80                         sol campestre  0.000000           0.000000   
81                           tecnolugico  0.000000           0.000000   
82                          torremolinos  0.000000           0.000000   
83                        villas del sol  0.000000           0.000000   
84          

In [24]:
dt_Merida_grouped.shape

(91, 216)

### Let's print each neighbourhood along with the top 5 most common venues

In [25]:
num_top_venues = 5

for hood in dt_Merida_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = dt_Merida_grouped[dt_Merida_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----alcal martin----
                venue  freq
0          Taco Place  0.25
1  Seafood Restaurant  0.25
2             Stadium  0.12
3         Coffee Shop  0.12
4  Mexican Restaurant  0.12


----altabrisa----
                 venue  freq
0       Clothing Store  0.08
1       Ice Cream Shop  0.05
2        Shopping Mall  0.05
3  Sporting Goods Shop  0.03
4          Coffee Shop  0.03


----benito jurez nte----
                 venue  freq
0        Movie Theater  0.08
1  American Restaurant  0.08
2           Steakhouse  0.08
3           Restaurant  0.08
4          Pizza Place  0.04


----buenavista----
                venue  freq
0          Restaurant  0.13
1   Convenience Store  0.13
2  Seafood Restaurant  0.07
3                 Spa  0.07
4         Bus Station  0.07


----camara de comercio norte----
                venue  freq
0      Ice Cream Shop  0.11
1  Mexican Restaurant  0.05
2         Karaoke Bar  0.05
3          Playground  0.05
4        Cocktail Bar  0.05


----campestre----
    

### Let's put that into a *pandas* dataframe

Sorting venues in descending order

In [26]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

### Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [27]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = dt_Merida_grouped['Neighbourhood']

for ind in np.arange(dt_Merida_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(dt_Merida_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head(10)

Neighbourhood 1st Most Common Venue 2nd Most Common Venue  \
0              alcal martin            Taco Place    Seafood Restaurant   
1                 altabrisa        Clothing Store        Ice Cream Shop   
2          benito jurez nte   American Restaurant         Movie Theater   
3                buenavista     Convenience Store            Restaurant   
4  camara de comercio norte        Ice Cream Shop     French Restaurant   
5                 campestre                Bakery            Taco Place   
6           carrillo ancona    Mexican Restaurant           Coffee Shop   
7         centro sct yucatn    Mexican Restaurant                  Park   
8                  cordemex        Clothing Store                 Hotel   
9      cumbres de altabrisa      Business Service            Eye Doctor   

  3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
0           Coffee Shop               Stadium            Food Stand   
1         Shopping Mall           Snack Place             Multiplex   
2            Steakhouse            Restaurant              Pharmacy   
3            Food Truck    Seafood Restaurant           Bus Station   
4          Soccer Field         Shopping Mall    Mexican Restaurant   
5    Mexican Restaurant           Flower Shop           Tailor Shop   
6     Convenience Store         Deli / Bodega        Clothing Store   
7                   Gym           Pizza Place                   Bar   
8        Ice Cream Shop             Multiplex        Cosmetics Shop   
9         Big Box Store     Convenience Store            Restaurant   

           6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
0             Mexican Restaurant  Yucatecan Restaurant                 Diner   
1                    Coffee Shop   American Restaurant   Sporting Goods Shop   
2                     Food Truck      Business Service                  Café   
3                            Spa            Taco Place          Dance Studio   
4                       Boutique           Karaoke Bar            Steakhouse   
5                 Sandwich Place                 Diner        Breakfast Spot   
6                         Bakery          Burger Joint              Pharmacy   
7           Arts & Entertainment        Gymnastics Gym        Ice Cream Shop   
8           Yucatecan Restaurant             Bookstore      Sushi Restaurant   
9  Vegetarian / Vegan Restaurant  Yucatecan Restaurant           Event Space   

  9th Most Common Venue 10th Most Common Venue  
0           Event Space          Event Service  
1            Shoe Store     Seafood Restaurant  
2    Italian Restaurant         Ice Cream Shop  
3           Supermarket   Fast Food Restaurant  
4       Motorcycle Shop       Swiss Restaurant  
5    Seafood Restaurant   Fast Food Restaurant  
6    Seafood Restaurant         Sandwich Place  
7    Seafood Restaurant             Taco Place  
8  Fast Food Restaurant    Sporting Goods Shop  
9         Event Service      Electronics Store

## 4. Cluster Neighbourhoods

Run *k*-means to cluster the neighbourhood into 10 clusters. we will be using k=10 as this is only for demostration on the Foursquare API and clustering, we are not analyzing the optimal k

In [28]:
# set number of clusters
kclusters = 10

dt_Merida_grouped_clustering = dt_Merida_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(dt_Merida_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([6, 1, 1, 1, 1, 2, 2, 6, 1, 1], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighbourhood.

In [29]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

dt_Merida_merged = newdf

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
dt_Merida_merged = dt_Merida_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

dt_Merida_merged.head() # check the last columns!

Postcode                  Neighbourhood        lat        lon  \
0     97000      jardines de san sebastian  20.989200 -89.756400   
1     97000            privada del maestro  20.982308 -89.626156   
2     97000                  merida centro  20.968927 -89.645942   
3     97000                      los cocos  20.948595 -89.630134   
4     97000  privada garcia gineres c - 29  20.989226 -89.638116   

   Cluster Labels          1st Most Common Venue 2nd Most Common Venue  \
0             NaN                            NaN                   NaN   
1             1.0  Paper / Office Supplies Store            Taco Place   
2             8.0                            Gym            Restaurant   
3             6.0                     Steakhouse    Mexican Restaurant   
4             2.0              Convenience Store    Mexican Restaurant   

   3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
0                    NaN                   NaN                   NaN   
1                  Hotel            Food Truck            Public Art   
2  Performing Arts Venue  Yucatecan Restaurant      Department Store   
3      Convenience Store          Dessert Shop            Laundromat   
4             Restaurant              Pharmacy  Fast Food Restaurant   

  6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
0                   NaN                   NaN                   NaN   
1           Men's Store    Mexican Restaurant                  Café   
2           Event Space         Event Service     Electronics Store   
3                  Park                   Bar            Taco Place   
4           Snack Place          Burger Joint    Seafood Restaurant   

  9th Most Common Venue 10th Most Common Venue  
0                   NaN                    NaN  
1       Bed & Breakfast                    Bar  
2            Donut Shop                Dog Run  
3    Athletics & Sports             Donut Shop  
4        Sandwich Place                    Bar

Finally, let's visualize the resulting clusters

In [30]:
#Drop NaN results
dt_Merida_merged.dropna(inplace=True)
dt_Merida_merged


Postcode                         Neighbourhood        lat        lon  \
1      97000                   privada del maestro  20.982308 -89.626156   
2      97000                         merida centro  20.968927 -89.645942   
3      97000                             los cocos  20.948595 -89.630134   
4      97000         privada garcia gineres c - 29  20.989226 -89.638116   
5      97003                             los reyes  20.978651 -89.575990   
6      97050                               yucatan  20.994835 -89.628827   
7      97050                          alcal martin  20.991941 -89.622019   
8      97059                              seoorial  20.990414 -89.621707   
9      97060                       carrillo ancona  20.985286 -89.643317   
10     97068                                itzaes  20.978626 -89.643890   
11     97069                            inalmbrica  20.982404 -89.647450   
12     97070                        dolores patron  20.989592 -89.626070   
13     97070                           el pedregal  20.995481 -89.634048   
14     97070                        garcia gineres  20.989879 -89.635259   
15     97080                             la huerta  20.983163 -89.615553   
16     97088                         santa cecilia  20.981542 -89.615670   
17     97089                               cupules  20.996456 -89.632023   
18     97098                               lourdes  20.960203 -89.610744   
19     97099                                 waspa  20.969501 -89.608684   
20     97100                             itzimna 2  20.992752 -89.614184   
21     97100                               itzimna  20.992752 -89.614184   
22     97100                     rinconada itzmina  20.992707 -89.606591   
23     97108                         las arboledas  20.981326 -89.608092   
24     97109                        jes䚂s carranza  20.986134 -89.612606   
25     97109                       ferrocarrileros  20.981507 -89.610899   
26     97113                             xaman-tan  21.033779 -89.604826   
27     97113                           san antonio  20.988620 -89.658706   
28     97113                            montebello  21.030032 -89.599605   
29     97113                  gran royal altabrisa  21.023328 -89.589453   
30     97113                         sol campestre  21.023322 -89.601552   
..       ...                                   ...        ...        ...   
64     97130                          privada maya  20.937700 -89.592300   
65     97130                             altabrisa  21.020171 -89.584606   
66     97130                  cumbres de altabrisa  21.026737 -89.588193   
67     97130                             missan ii  20.937700 -89.592300   
68     97130   residencial palmerales de altabrisa  21.028393 -89.590835   
69     97130                     santa rita cholul  21.032603 -89.576745   
70     97130                          torremolinos  21.027264 -89.587590   
71     97130                    vista alegre norte  21.026111 -89.577304   
73     97133  residencial camara de comercio norte  21.016379 -89.591297   
74     97133              camara de comercio norte  21.016379 -89.591297   
75     97133             los palmares de altabrisa  21.016717 -89.585574   
76     97133                            monterreal  21.020729 -89.601098   
77     97133                            montevideo  21.013900 -89.590779   
78     97134                                  maya  21.016717 -89.580189   
79     97134                  privada villa cholul  21.051270 -89.538662   
80     97134                  jose maria iturralde  21.023335 -89.572682   
81     97134                          paraiso maya  21.021330 -89.574119   
82     97135                    jardines de merida  21.000956 -89.594789   
83     97136            felipe carrillo puerto nte  21.009670 -89.623040   
84     97137                        mexico oriente  21.000362 -89.604794   
85     97138                           santa maria  

In [31]:
# create map
map_clusters = folium.Map(location=[20.97537, -89.61696], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(dt_Merida_merged['lat'], dt_Merida_merged['lon'], dt_Merida_merged['Neighbourhood'], dt_Merida_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5. Examining the Clusters

#### Cluster 0 - Sport oriented venues around

In [32]:
dt_Merida_merged.loc[dt_Merida_merged['Cluster Labels'] == 0, dt_Merida_merged.columns[[1] + list(range(5, dt_Merida_merged.shape[1]))]]

Neighbourhood 1st Most Common Venue 2nd Most Common Venue  \
36  residencial san angelo            Taco Place                 Diner   

   3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
36    Athletics & Sports        Baseball Field  Fast Food Restaurant   

   6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
36        Cosmetics Shop          Costume Shop    Falafel Restaurant   

   9th Most Common Venue 10th Most Common Venue  
36           Fabric Shop             Eye Doctor

#### Cluster 1 - Venues for High Income costumers & Tourism

In [33]:
dt_Merida_merged.loc[dt_Merida_merged['Cluster Labels'] == 1, dt_Merida_merged.columns[[1] + list(range(5, dt_Merida_merged.shape[1]))]]

Neighbourhood          1st Most Common Venue  \
1                    privada del maestro  Paper / Office Supplies Store   
12                        dolores patron                     Restaurant   
15                             la huerta                          Hotel   
16                         santa cecilia                          Hotel   
24                        jes䚂s carranza           Caribbean Restaurant   
25                       ferrocarrileros                      BBQ Joint   
26                             xaman-tan                  Shopping Mall   
27                           san antonio                      BBQ Joint   
28                            montebello             Italian Restaurant   
30                         sol campestre              Convenience Store   
31                           monte alban              Convenience Store   
32               privada monterreal plus              Convenience Store   
33                      hacienda dzodzil                     Restaurant   
34                              cordemex                 Clothing Store   
35                      gonzalo guerrero                    Coffee Shop   
37                         montes de ame                           Park   
38                     san antonio cucul                            Bar   
39                       san ramon norte                     Steakhouse   
42                    villas la hacienda                    Pizza Place   
43                      benito jurez nte            American Restaurant   
44                      gonzalo guerrero                    Coffee Shop   
49                  privada nuevo mexico             Mexican Restaurant   
51                            buenavista              Convenience Store   
53            residencial colonia mexico             Mexican Restaurant   
54                   privada real mexico             Mexican Restaurant   
55                          mexico norte             Mexican Restaurant   
56                   emiliano zapata nte                    Coffee Shop   
58                              san remo                            Spa   
59                            san carlos                    Coffee Shop   
63                            montecarlo                           Park   
65                             altabrisa                 Clothing Store   
66                  cumbres de altabrisa               Business Service   
69                     santa rita cholul                   Soccer Field   
70                          torremolinos           Gym / Fitness Center   
73  residencial camara de comercio norte                 Ice Cream Shop   
74              camara de comercio norte                 Ice Cream Shop   
75             los palmares de altabrisa                    Coffee Shop   
76                            monterreal              Convenience Store   
77                            montevideo                           Park   
78                                  maya                  Shopping Mall   
96                    jardines del norte             Mexican Restaurant   

   2nd Most Common Venue   3rd Most Common Venue 4th Most Common Venue  \
1             Taco Place                   Hotel            Food Truck   
12    Chinese Restaurant                   Hotel     Convenience Store   
15            Restaurant      Mexican Restaurant         Historic Site   
16        Ice Cream Shop             Coffee Shop            Restaurant   
24        Clothing Store            Optical Shop    Italian Restaurant   
25     Convenience Store  Furniture / Home Store            Restaurant   
26    Seafood Restaurant              Food Truck           Pizza Place   
27                   Gym               Pet Store           Post Office   
28                   Gym              Food Truck            Restaurant   
30                   Spa           Shopping Mall            Restaurant   
31    Seafood Restaurant                     Bar          Liqu

#### Cluster 2 - Restaurants, food venues mostly.

In [34]:
dt_Merida_merged.loc[dt_Merida_merged['Cluster Labels'] == 2, dt_Merida_merged.columns[[1] + list(range(5, dt_Merida_merged.shape[1]))]]

Neighbourhood 1st Most Common Venue  \
4   privada garcia gineres c - 29     Convenience Store   
8                        seoorial           Coffee Shop   
9                 carrillo ancona    Mexican Restaurant   
17                        cupules                 Diner   
19                          waspa               Theater   
20                      itzimna 2    Mexican Restaurant   
21                        itzimna    Mexican Restaurant   
22              rinconada itzmina    Mexican Restaurant   
45                    tecnolugico           Coffee Shop   
46                      campestre                Bakery   
47                      del norte                Bakery   
50                        montejo    Mexican Restaurant   
82             jardines de merida    Mexican Restaurant   
83     felipe carrillo puerto nte                Bakery   
84                 mexico oriente        Sandwich Place   

        2nd Most Common Venue          3rd Most Common Venue  \
4          Mexican Restaurant                     Restaurant   
8          Mexican Restaurant                     Restaurant   
9                 Coffee Shop              Convenience Store   
17             Sandwich Place  Paper / Office Supplies Store   
19               Dance Studio                    Flea Market   
20                        Bar             Italian Restaurant   
21                        Bar             Italian Restaurant   
22                 Restaurant              Electronics Store   
45          Convenience Store                    Pizza Place   
46                 Taco Place             Mexican Restaurant   
47  Middle Eastern Restaurant                    Pizza Place   
50                Pizza Place             Seafood Restaurant   
82                Pizza Place         Argentinian Restaurant   
83  Middle Eastern Restaurant                    Pizza Place   
84         Italian Restaurant                    Pizza Place   

   4th Most Common Venue 5th Most Common Venue   6th Most Common Venue  \
4               Pharmacy  Fast Food Restaurant             Snack Place   
8                Stadium           Beer Garden       Korean Restaurant   
9          Deli / Bodega        Clothing Store                  Bakery   
17     Electronics Store                   Bar              Food Truck   
19               Brewery            Taco Place                     Bar   
20                Bakery           Coffee Shop             Beer Garden   
21                Bakery           Coffee Shop             Beer Garden   
22            Taco Place                   Bar                Pharmacy   
45  Fast Food Restaurant                 Diner              Taco Place   
46           Flower Shop           Tailor Shop          Sandwich Place   
47     Electronics Store     College Cafeteria    Fast Food Restaurant   
50            Food Truck            Donut Shop                     Spa   
82         Shopping Mall     Mobile Phone Shop  Furniture / Home Store   
83     Electronics Store     College Cafeteria    Fast Food Restaurant   
84              Boutique    Mexican Restaurant                     Bar   

   7th Most Common Venue   8th Most Common Venue  \
4           Burger Joint      Seafood Restaurant   
8     Seafood Restaurant              Taco Place   
9           Burger Joint                Pharmacy   
17   Sporting Goods Shop  Furniture / Home Store   
19             BBQ Joint      Mexican Restaurant   
20     Convenience Store     Monument / Landmark   
21     Convenience Store     Monument / Landmark   
22      Sushi Restaurant          Sandwich Place   
45           Tailor Shop                  Bakery   
46                 Diner          Breakfast Spot   
47            Taco Place                   Diner   
50           Bus Station                 Stadium   
82                   Spa               Bookstore   
83            Taco Place                   Diner   
84                Bakery     Arts & Crafts Store   

            9th Most Common Venu

#### Cluster 3 - Big Box Stores

In [35]:
dt_Merida_merged.loc[dt_Merida_merged['Cluster Labels'] == 3, dt_Merida_merged.columns[[1] + list(range(5, dt_Merida_merged.shape[1]))]]

Neighbourhood 1st Most Common Venue 2nd Most Common Venue  \
29  gran royal altabrisa         Big Box Store            Eye Doctor   

   3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
29     Convenience Store            Food Truck  Yucatecan Restaurant   

   6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
29        Discount Store           Fabric Shop           Event Space   

   9th Most Common Venue 10th Most Common Venue  
29         Event Service      Electronics Store

#### Cluster 4 - Parks and recreational venues as movie theaters and shopping malls.

In [36]:
dt_Merida_merged.loc[dt_Merida_merged['Cluster Labels'] == 4, dt_Merida_merged.columns[[1] + list(range(5, dt_Merida_merged.shape[1]))]]

Neighbourhood 1st Most Common Venue 2nd Most Common Venue  \
41        villas del sol           Coffee Shop    Mexican Restaurant   
52   privada mediterrneo           Pizza Place                   Gym   
60  privada vista alegre            Taco Place         Movie Theater   
64          privada maya            Taco Place         Movie Theater   
67             missan ii            Taco Place         Movie Theater   

   3rd Most Common Venue 4th Most Common Venue          5th Most Common Venue  \
41     Electronics Store              Pharmacy                      Pet Store   
52                  Park            Taco Place                    Gaming Cafe   
60                  Park           Pizza Place  Paper / Office Supplies Store   
64                  Park           Pizza Place  Paper / Office Supplies Store   
67                  Park           Pizza Place  Paper / Office Supplies Store   

   6th Most Common Venue   7th Most Common Venue 8th Most Common Venue  \
41                  Park  Furniture / Home Store            Taco Place   
52         Movie Theater          Shipping Store                Bakery   
60    Basketball Stadium           Shopping Mall    Mexican Restaurant   
64    Basketball Stadium           Shopping Mall    Mexican Restaurant   
67    Basketball Stadium           Shopping Mall    Mexican Restaurant   

     9th Most Common Venue 10th Most Common Venue  
41             Tailor Shop          Movie Theater  
52             Music Venue               Pharmacy  
60  Furniture / Home Store             Sports Bar  
64  Furniture / Home Store             Sports Bar  
67  Furniture / Home Store             Sports Bar

##### Cluster 5 - Residential area, no parks around but a lot of Convenience Store.

In [37]:
dt_Merida_merged.loc[dt_Merida_merged['Cluster Labels'] == 5, dt_Merida_merged.columns[[1] + list(range(5, dt_Merida_merged.shape[1]))]]

Neighbourhood 1st Most Common Venue  \
68  residencial palmerales de altabrisa     Convenience Store   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
68  Yucatecan Restaurant                 Diner           Fabric Shop   

   5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
68            Eye Doctor           Event Space         Event Service   

   8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
68     Electronics Store            Donut Shop                Dog Run

#### Cluster 6 - Restaurants close to parks or gyms

In [38]:
dt_Merida_merged.loc[dt_Merida_merged['Cluster Labels'] == 6, dt_Merida_merged.columns[[1] + list(range(5, dt_Merida_merged.shape[1]))]]

Neighbourhood 1st Most Common Venue 2nd Most Common Venue  \
3                     los cocos            Steakhouse    Mexican Restaurant   
5                     los reyes    Seafood Restaurant     Convenience Store   
6                       yucatan     Convenience Store                   Bar   
7                  alcal martin            Taco Place    Seafood Restaurant   
10                       itzaes                Bakery     Convenience Store   
11                   inalmbrica            Taco Place           Pizza Place   
13                  el pedregal    Mexican Restaurant     Convenience Store   
14               garcia gineres     Convenience Store            Food Truck   
18                      lourdes              Boutique     Convenience Store   
23                las arboledas     Convenience Store        Baseball Field   
40                plan de ayala                  Pool            Salsa Club   
48            centro sct yucatn    Mexican Restaurant                  Park   
57                 vista alegre            Taco Place            Restaurant   
62                   diaz ordaz    Mexican Restaurant           Comedy Club   
71           vista alegre norte     Convenience Store    Mexican Restaurant   
79         privada villa cholul                  Farm        History Museum   
80         jose maria iturralde    Mexican Restaurant            Taco Place   
81                 paraiso maya    Mexican Restaurant     Convenience Store   
85                  santa maria        Sandwich Place            Taco Place   
86             san pedro cholul    Mexican Restaurant                  Park   
87                    los pinos    Mexican Restaurant      Sushi Restaurant   
93                      el arco            Taco Place              Pharmacy   
94  jardines de vista alegre ii    Mexican Restaurant            Taco Place   
95     jardines de vista alegre            Taco Place    Mexican Restaurant   
97         residencial del arco     Convenience Store            Taco Place   

   3rd Most Common Venue      4th Most Common Venue  \
3      Convenience Store               Dessert Shop   
5             Taco Place                 Restaurant   
6             Food Truck                 Taco Place   
7            Coffee Shop                    Stadium   
10            Taco Place         Seafood Restaurant   
11                  Park               Cupcake Shop   
13                 Diner        Sporting Goods Shop   
14            Restaurant       Fast Food Restaurant   
18                  Park    Health & Beauty Service   
23                   Gym     Furniture / Home Store   
40           Snack Place                 Taco Place   
48                   Gym                Pizza Place   
57              Pharmacy         Italian Restaurant   
62    Seafood Restaurant                       Café   
71            Taco Place                        Gym   
79            Taco Place                 Food Truck   
80     Convenience Store                        Gym   
81                   Gym  Middle Eastern Restaurant   
85                  Park         Mexican Restaurant   
86       Thai Restaurant              Grocery Store   
87     Convenience Store                 Taco Place   
93            Restaurant         Italian Restaurant   
94    Italian Restaurant          Convenience Store   
95     Convenience Store                Coffee Shop   
97    Mexican Restaurant         Seafood Restaurant   

        5th Most Common Venue      6th Most Common Venue  \
3                  Laundromat                       Park   
5                        Park         Mexican Restaurant   
6              Sandwich Place                      Diner   
7                  Food Stand         Mexican Restaurant   
10  Cajun / Creole Restaurant                    Stadium   
11               Skating Rink                 Restaurant   
13                       Park         Seafood Restaurant   
14                Comedy Club               Dessert 

#### Cluster 7 - Stables around, outside the city.

In [39]:
dt_Merida_merged.loc[dt_Merida_merged['Cluster Labels'] == 7, dt_Merida_merged.columns[[1] + list(range(5, dt_Merida_merged.shape[1]))]]

Neighbourhood 1st Most Common Venue 2nd Most Common Venue  \
98  vista alegre lotificacion               Stables                Buffet   

   3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
98                  Pool                   Gym         Grocery Store   

   6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
98          Gourmet Shop           Event Space         Event Service   

   9th Most Common Venue 10th Most Common Venue  
98     Electronics Store             Donut Shop

#### Cluster 8 - Highly touristic places and gyms as the most common venue.

In [40]:
dt_Merida_merged.loc[dt_Merida_merged['Cluster Labels'] == 8, dt_Merida_merged.columns[[1] + list(range(5, dt_Merida_merged.shape[1]))]]

Neighbourhood 1st Most Common Venue 2nd Most Common Venue  \
2  merida centro                   Gym            Restaurant   

   3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
2  Performing Arts Venue  Yucatecan Restaurant      Department Store   

  6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
2           Event Space         Event Service     Electronics Store   

  9th Most Common Venue 10th Most Common Venue  
2            Donut Shop                Dog Run

#### Cluster 9 - Outside the city with sports club as the most common venue.

In [41]:
dt_Merida_merged.loc[dt_Merida_merged['Cluster Labels'] == 9, dt_Merida_merged.columns[[1] + list(range(5, dt_Merida_merged.shape[1]))]]

Neighbourhood 1st Most Common Venue 2nd Most Common Venue  \
99  san antonio cinta           Sports Club              Pharmacy   

   3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
99         Auto Workshop   Fried Chicken Joint          Dessert Shop   

   6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
99           Fabric Shop            Eye Doctor           Event Space   

   9th Most Common Venue 10th Most Common Venue  
99         Event Service      Electronics Store

# Results and discussion<br>

### I decided to use the  first 100 postal codes from Merida for this excercise due to their exposure in Foursquare as the people found there are most likely to utilice this application for tips and reviews. A brief expected behaviour for each cluster is written next to it.  

### The clusters were defined by the most common type of venue: <br>
   <li> <b>Cluster 0, Sport oriented venues around:</b>   If the places remain closed for activities as expected, it should show moderate economic downturn and represent low risk of contagion.<br></li>
   <li> <b>Cluster 1, Venues for High Income costumers:</b> Common places known for nightlife, economic downturn and expected low risk of contagion as all this venues are closed. <br></li>
   <li> <b>Cluster 2, Restaurants, food venues mostly: </b> High economic downturn, probably most of the venues will received a hard hit on their operations and cashflow, expected shutdown of the smallest venues of this cluster. Low risk of contagion<br></li>
   <li> <b>Cluster 3, Big Box Stores:</b> Places still opened due to their food and basic needs distribution function. The risk of contagions its moderate as people gather for buying.<br></li>
   <li> <b>Cluster 4, Parks and recreational venues as movie theaters and shopping mall:</b> High economic downturn, low risk of contagion.<br></li>
   <li> <b>Cluster 5, Residential area, no parks around but a lot of Convenience Store:</b> Expected economic downturn and moderate contagion risk for the people gathering on the convenience stores. <br></li>
   <li> <b>Cluster 6, Restaurants close to parks or gyms:</b> High economic downturn, low risk of contagion as the gyms remain closed.<br></li>
   <li> <b>Cluster 7, Stables around, outside the city:</b> Expected economic downturn, low risk of contagion.<br></li>
   <li> <b>Cluster 8, Highly touristic places and gyms as the most common venue:</b> Very high economic downturn caused by lack of international tourism, food venues almost closed, moderate risk of contagion<br></li>
   <li> <b>Cluster 9, Outside the city with sports club as the most common venue:</b> Expected economic downturn, low risk of contagion.<br></li>
   

# Conclussion<br>
 In conclusion, we can observe that, regardless of the cluster, the most often venue are the restaurants. We should look for special importance to this as this kind of venue shows that it receives the most economic damage during this pandemic.</br> In Mexico, 97% of food related venues are classified as micro or small companies within 10 or fewer employees (CANIRAC, 2014), this means that they are an economic sector highly affected by situations such as COVID-19's outbreak. </br> As society, we and government should take special care for this business sector in an attempt to stop the disappearance of jobs created by restaurant entrepreneurs.

## Bibliografy
https://molekule.science/places-to-avoid-flu-virus/ <br>
https://www.babymed.com/health-news/8-public-places-avoid-during-cold-and-flu-season <br>
https://www.nhs.uk/conditions/coronavirus-covid-19/ <br>
https://www.health.gov.au/news/health-alerts/novel-coronavirus-2019-ncov-health-alert/what-you-need-to-know-about-coronavirus-covid-19 <br>
https://www.who.int/emergencies/diseases/novel-coronavirus-2019/advice-for-public <br>
https://www.healthline.com/health-news/public-places-and-the-coronavirus-what-to-know#Coronavirus-can-spread-through-contact-with-contaminated-surfaces,-too <br>
https://www.cdc.gov/coronavirus/2019-ncov/prepare/transmission.html <br>
https://www.bbc.com/future/article/20200317-covid-19-how-long-does-the-coronavirus-last-on-surfaces <br>
https://canirac.org.mx/images//files/TODO%20SOBRE%20LA%20MESA%20ESTUDIOS%20DE%20LA%20INDUSTRIA.pdf

This notebook was <b>The final Capstone</b> from the week 5 of the Applied Data Science Capstone track from IBM Professional Certificate made by <a href='https://www.linkedin.com/in/novelo-luis/'> Luis Novelo </a>

***

***
